### Step1 导入相关包

In [1]:
import os

from langchain_community.llms import BaichuanLLM
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

from langchain_core.documents import Document
from langchain_core.retrievers import BaseRetriever
from langchain_core.callbacks import CallbackManagerForRetrieverRun
from typing import List
from sentence_transformers import CrossEncoder

d:\miniconda3\envs\transformers\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Step2 加载数据

In [2]:
data = "../demo/"
# 这里为了做演示，随便找了一个pdf

In [3]:
loader = PyPDFDirectoryLoader(data)

docs_before_split = loader.load()
# 过滤目录和附录
docs_before_split = [doc for doc in docs_before_split if doc.metadata['page'] > 7 and doc.metadata['page'] <275] 
# 这里可以理解为对自己的文档切分成块，chuck size是每一块的大小，可以根据需求调整
# text_splitter = RecursiveCharacterTextSplitter(
#     chunk_size = 256,
#     chunk_overlap  = 30,
# )
# docs_after_split = text_splitter.split_documents(docs_before_split)

# docs_after_split[0]
docs_before_split[0]

Document(page_content='第一章 电力现货市场基础   \n             \n1   \n第一章 \n \n \n电力现货市场基础  \n \n \n \n  \n 1. 什么是电力市场？电力市场与 普通商品市场有哪 些差异？电力市场有\n哪些特征？ \n（1）电力市场的概念。  \n我国关于电力市场的权威解释始见于《中国电力百科全书  电力系统卷（第二版） 》。\n电力市场的定义为：基于市场经济原则，电力市场的定义为基于市场经济原则，为实现\n电力商品交换的电力工业组织结构、经营管理和运行规则的总和。电力市场又是一个具体的执行系统，包括交易场所、交易管理系统、计量和结算系统、信息和通信系统等。\n上面从组织和实操两个维度对电力市场进行了描绘。  \n通常电力市场包括广义和狭义两种含义。比照商品市场的一般定义，广义的电力市场\n泛指电力流通交换的领域。按照该含义，自电力作为商品实现交换之日起，电力市场就\n已经存在。当今在涉及市场规模和范围的语境下，运用的便是广义电力市场的概念。广\n义的电力市场有着明确的地域和容量指向，电力市场的地理边界可能差异很大，例如单一州（省）域内的电力市场，或者整个国家的电力市场，乃至跨国的电力市场。而这种\n地理边界受限于电网的覆盖范围，由于电网是电力传输的唯一通道，故多大范围的电网\n才可能有多大范围的电力市场；电力市场容量也同样存在很大的差异。  \n狭义的电力市场是指现代竞争性的电力市场， 《中国电力百科全书》定义的电力市场\n即为狭义的电力市场，建立电力市场旨在通过开放、竞争等市场手段实现电力能源资源\n的优化配置。所谓基于市场经济原则主要是指电能生产者和使用者本着公平竞争、自愿互利的原则，通过协商、竞价等方式，就电能及其相关产品进行交易，通过市场竞争确\n定价格和数量的市场原则。目前广泛讨论的电力市场，除特殊语境下，通常都是指狭义\n的电力市场。  \n之所以称之为狭义的电力市场，主要原因有二。  \n1）专指现代电力市场，狭义的电力市场兴起于 20世纪 80年代，是在电力作为商品\n出现的百年之后，而这百年间，广义电力市场的形态发生了曲折的变迁。电力工业初创', metadata={'source': '..\\demo\\电力现货市场101问.pdf', 'page': 8})

In [4]:
# 提取所有文本
full_text=""
for doc in docs_before_split:
    full_text += doc.page_content

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 256,
    chunk_overlap  = 30,
)
docs_after_split = text_splitter.create_documents([full_text])
docs_after_split = [Document(page_content=doc.page_content,  metadata={"source": "..\demo\电力现货市场101问.pdf"}) for doc in docs_after_split]
docs_after_split[0]
# # 按段落分割
# paragraphs = full_text.split('。  \n')
# #去除段落中的空格和换行
# paragraphs = [paragraph.replace('\n','').replace(' ','') for paragraph in paragraphs]
# #去除一些无用段落
# paragraphs = [paragraph for paragraph in paragraphs if len(paragraph)>50]
# docs_after_split = [Document(page_content=doc,  metadata={"source": "..\demo\电力现货市场101问.pdf"}) for doc in paragraphs]

# docs_after_split[0]

Document(page_content='第一章 电力现货市场基础   \n             \n1   \n第一章 \n \n \n电力现货市场基础  \n \n \n \n  \n 1. 什么是电力市场？电力市场与 普通商品市场有哪 些差异？电力市场有\n哪些特征？ \n（1）电力市场的概念。  \n我国关于电力市场的权威解释始见于《中国电力百科全书  电力系统卷（第二版） 》。\n电力市场的定义为：基于市场经济原则，电力市场的定义为基于市场经济原则，为实现', metadata={'source': '..\\demo\\电力现货市场101问.pdf'})

### Step3 创建向量数据库

In [5]:
# 从过往工作经验看，embedding对于rag效果影响比较大，一般首选还是openai embedding做这一部分，开源的效果很一般
huggingface_embeddings = HuggingFaceBgeEmbeddings(
    model_name="moka-ai/m3e-base",  # 使用m3e模型做embeddding
    model_kwargs={'device':'cpu'},
    encode_kwargs={'normalize_embeddings': True}
)

d:\miniconda3\envs\transformers\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [6]:
vectorstore = FAISS.from_documents(docs_after_split, huggingface_embeddings)

### Step4 创建QA链

In [7]:
query = """什么是电力市场？"""
         # Sample question, change to other questions you are interested in.
relevant_documents = vectorstore.similarity_search(query,k=25)
for i, doc in enumerate(relevant_documents):
    print(f"检索到的第{i+1}个内容: \n {doc.page_content}", end="\n-----------------------------------------------------\n")


# 从这一步看，已经很清晰rag的原理了，即从大量文件索引找到top k相关的text块，供下一步LLM查找、总结答案

检索到的第1个内容: 
 市场”的概念，即在狭义电力市场的基础上，将电力市场化改革领域拓展到规划、投融资、碳交易、环境约束等环节，以实现电力生产、交换相关上下游环节的全链条市场化。  
电力市场作为电力工业市场化改革的目标模式，因其改革的动因和目标取向不同，以
及市场外部环境的差异，呈现出显著的多样性，也使其结构显得格外复杂。世界各地的电力市场，既没有统一的标准供引用，也没有通用的模板可复制，需要结合各自的国情、
民情和网情建设和运营。总体上，电力市场有批发市场（ wholesale market ）与零售市场
-----------------------------------------------------
检索到的第2个内容: 
 义的电力市场有着明确的地域和容量指向，电力市场的地理边界可能差异很大，例如单一州（省）域内的电力市场，或者整个国家的电力市场，乃至跨国的电力市场。而这种
地理边界受限于电网的覆盖范围，由于电网是电力传输的唯一通道，故多大范围的电网
才可能有多大范围的电力市场；电力市场容量也同样存在很大的差异。  
狭义的电力市场是指现代竞争性的电力市场， 《中国电力百科全书》定义的电力市场
即为狭义的电力市场，建立电力市场旨在通过开放、竞争等市场手段实现电力能源资源
-----------------------------------------------------
检索到的第3个内容: 
 市场由发电企业处购买电能，再通过电力零售市场向终端用户出售电力商品。  
（2）市场性质。  
电力市场按其市场性质可分为实物市场与金融市场。一般而言，实物市场与金融市场
可以通过按产品类型和市场主体的意图两个方面加以辨识。电力实物市场建设几乎是各
国各地电力市场建设的重心，建设运营中普遍接受电力行政主管部门或监管机构的监管；电力金融市场严格意义上要接受金融监管机构的监管。  
1）电力实物市场。  
实物市场，业界也有译之为物理市场（ physical market ）的，它是以电能量及其相关
-----------------------------------------------------
检索到的第4个内容: 
 市场的功能及特点，具体如下。               电力现货市场 101 问  
  42 
 
图1 − 4 现货

In [8]:
from sentence_transformers import CrossEncoder

cross_encoder = CrossEncoder(
    "BAAI/bge-reranker-base", max_length=512, device="cpu"
)
reranked_docs = cross_encoder.rank(
        query,
        [doc.page_content for doc in relevant_documents],
        top_k=5,
        return_documents=True,)
reranked_docs

[{'corpus_id': 1,
  'score': 0.9989737,
  'text': '义的电力市场有着明确的地域和容量指向，电力市场的地理边界可能差异很大，例如单一州（省）域内的电力市场，或者整个国家的电力市场，乃至跨国的电力市场。而这种\n地理边界受限于电网的覆盖范围，由于电网是电力传输的唯一通道，故多大范围的电网\n才可能有多大范围的电力市场；电力市场容量也同样存在很大的差异。  \n狭义的电力市场是指现代竞争性的电力市场， 《中国电力百科全书》定义的电力市场\n即为狭义的电力市场，建立电力市场旨在通过开放、竞争等市场手段实现电力能源资源'},
 {'corpus_id': 0,
  'score': 0.99877423,
  'text': '市场”的概念，即在狭义电力市场的基础上，将电力市场化改革领域拓展到规划、投融资、碳交易、环境约束等环节，以实现电力生产、交换相关上下游环节的全链条市场化。  \n电力市场作为电力工业市场化改革的目标模式，因其改革的动因和目标取向不同，以\n及市场外部环境的差异，呈现出显著的多样性，也使其结构显得格外复杂。世界各地的电力市场，既没有统一的标准供引用，也没有通用的模板可复制，需要结合各自的国情、\n民情和网情建设和运营。总体上，电力市场有批发市场（ wholesale market ）与零售市场'},
 {'corpus_id': 24,
  'score': 0.998711,
  'text': '于降低发电煤耗，有利于充分利用清洁能源，有利于引导用户合理消费电能。  \n电力市场是电力工业市场化改革的产物，迄今为止，所有的电力市场都处于不断完善\n的进程中，所以对电力市场的认识也需要与时俱进。  \n 2. 什么是电力市场体系？电力现 货市场在电力市场 体系中的地位和作用\n是什么？ \n（1）电力市场体系的概念。  \n一般而言，电力市场体系是由政府监管下各类电力细分市场有机组合而成的群体系'},
 {'corpus_id': 10,
  'score': 0.9983376,
  'text': '电力商品交换的电力工业组织结构、经营管理和运行规则的总和。电力市场又是一个具体的执行系统，包括交易场所、交易管理系统、计量和结算系统、信息和通信系统等。\n上面从组织和实操两个维度对电力市场进行了描绘。

In [30]:
# 创建检索器
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 5})

In [15]:

""" 使用自定义的检索器,重写_get_relevant_documents方法来实现对相似性检索召回的文本块进行重排"""
class CustomRetriever(BaseRetriever):
    
    def _get_relevant_documents(
        self, query: str, *, run_manager: CallbackManagerForRetrieverRun
    ) -> List[Document]:
        cross_encoder = CrossEncoder("BAAI/bge-reranker-base", max_length=512, device="cpu")
        relevant_documents = vectorstore.similarity_search(query,k=25)
        reranked_docs = cross_encoder.rank(
        query,
        [doc.page_content for doc in relevant_documents],
        top_k=5,
        return_documents=True,)

        reranking_relevant_documents= []
        for doc in reranked_docs:
           reranking_relevant_documents.append(
               Document(page_content=doc["text"],
                        metadata=relevant_documents[doc["corpus_id"]].metadata)
                        ) 
        return reranking_relevant_documents

In [16]:
# 自定义加入重排的检索器
retriever = CustomRetriever()

In [31]:
# 创建prompt
prompt_template = """
请参考下面的相关文本回答问题，如果不知道答案，就回复不知道，

{context}

Question: {question}

Helpful Answer:
"""

prompt = PromptTemplate(
 template=prompt_template, input_variables=["context", "question"]
)

In [32]:
# 创建百川的LLM
llm = BaichuanLLM()

In [33]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt}
)

### Step5 提问，运行QA链，得到RAG结果

In [46]:
question = "什么是电力市场？"
# question = "电力辅助服务市场与电力现货市场的关系？"
result = qa_chain({"query": question})
# result["result"]
print("========= chain result ==========")
print(result['result'])

========= chain result ==========
电力市场是一种概念，它是指在狭义电力市场的基础上，将电力市场化改革领域拓展到规划、投融资、碳交易、环境约束等环节，以实现电力生产、交换相关上下游环节的全链条市场化。电力市场作为电力工业市场化改革的目标模式，因其改革的动因和目标取向不同，以及市场外部环境的差异，呈现出显著的多样性，也使其结构显得格外复杂。


In [44]:
relevant_docs = result['source_documents']
print(f'There are {len(relevant_docs)} documents retrieved which are relevant to the query.')
print("*" * 100)
for i, doc in enumerate(relevant_docs):
    # print(f"Relevant Document #{i+1}:\nSource file: {doc.metadata['source']}, Page: {doc.metadata['page']}\nContent: {doc.page_content}")
    print(f"Relevant Document #{i+1}:\nSource file: {doc.metadata['source']}\nContent: {doc.page_content}")
    print("-"*100)
    print(f'There are {len(relevant_docs)} documents retrieved which are relevant to the query.')

There are 5 documents retrieved which are relevant to the query.
****************************************************************************************************
Relevant Document #1:
Source file: ..\demo\电力现货市场101问.pdf
Content: 市场”的概念，即在狭义电力市场的基础上，将电力市场化改革领域拓展到规划、投融资、碳交易、环境约束等环节，以实现电力生产、交换相关上下游环节的全链条市场化。  
电力市场作为电力工业市场化改革的目标模式，因其改革的动因和目标取向不同，以
及市场外部环境的差异，呈现出显著的多样性，也使其结构显得格外复杂。世界各地的电力市场，既没有统一的标准供引用，也没有通用的模板可复制，需要结合各自的国情、
民情和网情建设和运营。总体上，电力市场有批发市场（ wholesale market ）与零售市场
----------------------------------------------------------------------------------------------------
There are 5 documents retrieved which are relevant to the query.
Relevant Document #2:
Source file: ..\demo\电力现货市场101问.pdf
Content: 义的电力市场有着明确的地域和容量指向，电力市场的地理边界可能差异很大，例如单一州（省）域内的电力市场，或者整个国家的电力市场，乃至跨国的电力市场。而这种
地理边界受限于电网的覆盖范围，由于电网是电力传输的唯一通道，故多大范围的电网
才可能有多大范围的电力市场；电力市场容量也同样存在很大的差异。  
狭义的电力市场是指现代竞争性的电力市场， 《中国电力百科全书》定义的电力市场
即为狭义的电力市场，建立电力市场旨在通过开放、竞争等市场手段实现电力能源资源
------------------------------------------------------------

### Step6 开始评估

In [193]:
#result['ground_truths'] = "电力批发市场按其市场属性，可分为电力实物市场与电力金融市场。一般而言，电力实物市场与电力金融市场可以通过按产品类型和市场主体的意图两个方面加以辨识。电力实物市场建设几乎是各国各地电力市场建设的重心，建设运营中普遍接受电力监管机构的监管；电力金融市场严格意义上要接受金融监管机构的监管（见问题 3）。电力金融市场涉及能源电力衍生出的金融产品的交易行为，具有金融衍生属性，包括市场结构与相关的制度安排、市场主体、产品与交易，同时也具备其特有的供求驱动因素。电力金融市场合同通常不涉及电力实物商品的交割，取而代之的是现金的交割。电力金融市场中，一般参照金融市场期货、期权交易的基本原理进行电力期货、电力期权等电力金融衍生产品交易，其交易的对象是电力金融衍生品。"

from ragas.metrics import faithfulness, answer_relevancy, context_relevancy
from ragas.langchain.evalchain import RagasEvaluatorChain

# make eval chains
eval_chains = {
    m.name: RagasEvaluatorChain(metric=m) 
    for m in [faithfulness, answer_relevancy, context_relevancy]
}

# evaluate
for name, eval_chain in eval_chains.items():
    score_name = f"{name}_score"
    print(f"{score_name}: {eval_chain(result)[score_name]}")


faithfulness_score: 1.0
answer_relevancy_score: 0.8901393929213829
context_relevancy_score: 0.14893617021276595
